In [251]:
import tweepy
from tweepy import Stream, OAuthHandler, StreamListener
import json
import time
from collections import Counter
import re
import pandas as pd
from langdetect import detect
import pyprind
import deepdish as dd

In [252]:
#import secret codes
from twitter_pwd import access_token, access_token_secret, consumer_key, consumer_secret

In [253]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

##### Get random tweets from a given coordinate box

In [415]:
# 
data_list, texts, langs, locs = [], [], [], []

class StdOutListener(StreamListener):
    """ A listener handles tweets are the received from the stream.
    This is a basic listener that just prints received tweets to stdout.
    """
    
    def __init__(self):
        self.data_list = []
        self.texts = []
        self.langs = []
    
    def on_data(self, data):
        jd = json.loads(data)
        self.data_list.append(jd)
        self.texts.append(jd['text'])
        self.langs.append(jd['lang'])
        try:
            print(data)
            saveFile = open('newtweets.csv', 'a')
            saveFile.write(data).encode("utf8")
            saveFile.write('/n').encode("utf8")
            saveFile.close()
            return True
        except BaseException:
            print ('failed ondata')
            time.sleep(5)

    def on_error(self, status):
        print(status)

In [416]:
#coordinates
Lviv = [23.882904,49.763526,24.163055,49.921167]
Kiev = [30.449982,50.408518,30.639496,50.495958]
Yerevan = [44.329834,40.078071,44.681396,40.296287]
Brussel = [4.258575,50.788575,4.489288,50.913424]
Barcelona = [1.835403,41.375778,2.241898,41.586688]

In [10]:
# #Barcelona
# l_Barc = StdOutListener()
# #ASK FOR KEYWORD TO COLLECT DATA
# stream_Barc = Stream(auth, l_Barc)
# stream_Barc.filter(locations=Barcelona)

In [9]:
#Counter(l_Barc.langs)
# for data, lang in zip(l_Barc.data_list, l_Barc.langs):
#     print(data['user']['location'], lang)
for text in l_Barc.texts:
    if type(text) == str:
        print(text)
    else:
        print(text.decode('utf-8'))

In [10]:
# for ee in l_Barc.data_list:
#     print(ee['place']['id'], ee['place']['place_type'])

In [11]:
# #Brussel
# l_Bru = StdOutListener()
# #ASK FOR KEYWORD TO COLLECT DATA
# stream_Bru = Stream(auth, l_Bru)
# stream_Bru.filter(locations=Brussel)

In [12]:
# #Counter(l_Bru.langs)
# for data, lang in zip(l_Bru.data_list, l_Bru.langs):
#     print(data['user']['location'], lang)

In [13]:
# #LVIV
# l_Lv = StdOutListener()
# #ASK FOR KEYWORD TO COLLECT DATA
# stream_Lv = Stream(auth, l_Lv)
# stream_Lv.filter(locations=Lviv)

In [14]:
# #YEREVAN
# l_Yer = StdOutListener()
# #ASK FOR KEYWORD TO COLLECT DATA
# stream_Yer = Stream(auth, l_Yer)
# stream_Yer.filter(locations=Yerevan)

In [15]:
# for data, lang in zip(l_Yer.data_list, l_Yer.langs):
#     print(data['user']['location'], lang)

In [16]:
# for text in l_Yer.texts:
#     if type(text) == str:
#         print(text)
#     else:
#         print(text.decode('utf-8'))

In [17]:
# for data, lang in zip(data_list, langs):
#     print(data['user']['location'], lang)

In [18]:
#KIEV
l_Kiev = StdOutListener()
#ASK FOR KEYWORD TO COLLECT DATA
stream_Kiev = Stream(auth, l_Kiev)
stream_Kiev.filter(locations=Kiev)

In [19]:
#Counter(l_Kiev.langs)
# for data, lang in zip(l_Kiev.data_list, l_Kiev.langs):
#     print(data['user']['location'], lang)
for text in l_Kiev.texts:
    if type(text) == str:
        print(text)
    else:
        print(text.decode('utf-8'))

In [20]:
for data in l_Kiev.data_list:
    print(data['place']['id'], data['place']['place_type'], 
          data['place']['country'], data['user']['location'])

In [125]:
for text in texts:
    if type(text) == str:
        print(text)
    else:
        print(text.decode('utf-8'))


### KEY FUNCTIONS : USERS, FOLLOWERS, TIMELINES, LANGS

In [255]:
import tweepy
import time
import re

def get_account_network(account_name, rel_type='followers', max_num =100, key_words=None):
    pbar = pyprind.ProgBar(max_num)
    list_people = []
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    users = tweepy.Cursor(getattr(api, rel_type, 0), screen_name=account_name).items(max_num)
    while True:
        try:
            user = next(users)
            if not key_words:
                list_people.append(user)
            else:
                locs = '|'.join(key_words)
                patt = re.compile(locs)
                found_loc = re.findall(patt, user._json['location'])
                if found_loc:
                    list_people.append(user)
        except tweepy.TweepError as e:
            if 'Read timed out' in str(e):
                print('fallen here')
                print(e)
                time.sleep(5)
            else:
                time.sleep(60*16)
                user = next(users)
        except StopIteration:
            break            
        pbar.update()
    return list_people
        

In [241]:
def get_account_timeline(account_name, max_num_twts=10):
    """ Given an account name,
        it retrieves a maximum number of tweets stored in a list
        Args:
            * account name: string. Screen_name that identifies the twitter account
            * max_num_twts: integer. Maximum number of tweets to be retrieved for each account
        Returns:
            * list_tweets: list of strings including all retrieved tweets"""
    list_tweets=[]
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    timeline = tweepy.Cursor(api.user_timeline, screen_name=account_name).items(max_num_twts)
    i=0
    while True:
        try:
            tw = next(timeline)
            list_tweets.append(tw)
        except tweepy.TweepError as e:
            if '401' in str(e):    
                print(e)
                time.sleep(10)
                break
            else:
                time.sleep(60*15)
                tw = next(timeline)
        except StopIteration:
            break
    return list_tweets

In [242]:
def get_twts_from_list_account_names(list_accounts, max_num_accounts=None, max_num_twts=10):
    pbar = pyprind.ProgBar(len(list_accounts))
    texts_tweets = []
    langs_tweets = []
    authors_tweets = []
    if max_num_accounts:
        list_accounts = list_accounts[:max_num_accounts]
    for idx, f in enumerate(list_accounts):
        #print(idx)
        tl = get_account_timeline(f, max_num_twts=max_num_twts)
        texts_tweets.extend([tw.text for tw in tl])
        langs_tweets.extend([tw.lang for tw in tl])
        authors_tweets.extend([f for _ in tl])
        pbar.update()
    return texts_tweets, langs_tweets, authors_tweets

### UKRAINE: data structure and relevant twitter accounts 

In [12]:
Ukraine_nodes = {}
Ukraine_nodes['cities'] = ['kiev', 'odessa', 'lviv', 'kharkov', 'dnipropetrovsk']
Ukraine_nodes['city_sites'] = {'Mariupol':['0629ComUa'], 
                               'kiev':['kievtypical','kliniki_kiev','LISOD_clinic','avto_kiev', 'editbeauty']}
Ukraine_nodes['news'] = ['HromadskeUA','tsnua','ukrpravda_news', 'lb_ua', 'Korrespondent', 
                         'Delo_ua', 'BBC_ua', 'LIGAnet', 'segodnya_life']
Ukraine_nodes['TV'] = ['5channel', 'EspresoTV', '24tvua', 'footballua_tv']
Ukraine_nodes['starsystem'] = ['VeraBrezhneva', 's_vakarchuk', 'KAMEHCKUX']

In [14]:
key_words=['Україна', 'Ukraine', 'Украина', 'Київ', 'Киев']
HromadskeUA_followers = get_account_network('HromadskeUA', rel_type='followers', 
                                            max_num =5000, key_words=key_words)

In [256]:
country = 'ukr'
node_name = 'starsystem'
acc_name = 's_vakarchuk'
rel_type = 'followers'

#key_words=['Україна', 'Ukraine', 'Украина', 'Київ', 'Киев']
path_save = '/'.join(['',country, node_name, acc_name, rel_type])
followers = get_account_network(acc_name, rel_type=rel_type, 
                                max_num =5000, key_words=None)
json_format = [elem._json for elem in followers]
df = pd.DataFrame(json_format)
df.to_hdf('lang_data.h5', path_save)

0%                          100%
[#                             ] | ETA: 00:01:56

Rate limit reached. Sleeping for: 895
fallen here

[###                           ] | ETA: 02:20:50


Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=60)
Rate limit reached. Sleeping for: 896
fallen here

[#####                         ] | ETA: 02:35:48

...


Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=60)
Rate limit reached. Sleeping for: 894
fallen here

[############################  ] | ETA: 00:16:52


Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=60)
Rate limit reached. Sleeping for: 892
fallen here

[##############################] | ETA: 00:00:00
Total time elapsed: 04:12:35

Total time elapsed: 04:12:35

Total time elapsed: 04:12:35

Total time elapsed: 04:12:35

Total time elapsed: 04:12:35

Total time elapsed: 04:12:35

Total time elapsed: 04:12:35

Total time elapsed: 04:12:35

Total time elapsed: 04:12:35

Total time elapsed: 04:12:35

Total time elapsed: 04:12:35

Total time elapsed: 04:12:35

Total time elapsed: 04:12:35

Total time elapsed: 04:12:35

Total time elapsed: 04:12:35

Total time elapsed: 04:12:35

Total time elapsed: 04:12:35
//anaconda/lib/python3.5/site-packages/pandas/core/generic.py:939: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->['created_at', 'description', 'entities', 'id_str', 'lang', 'location', 'name', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https', 'profile_banner_url', 


Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=60)


In [210]:
acc_name = 'LIGAnet'
df = pd.read_hdf('lang_data.h5', '/ukr_nodes/news/' + acc_name + '/followers')

In [196]:
df['lang'][df['location'].str.contains(r"(Україна|Ukraine|Украина|Київ|Киев)")].value_counts()

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


ru    420
uk    154
en     21
it      1
Name: lang, dtype: int64

In [197]:

df[df['statuses_count'] >= 5][['screen_name', 'statuses_count','followers_count']].count()

screen_name        501
statuses_count     501
followers_count    501
dtype: int64

In [78]:
#HRMUA_flwrs = [f for idx, f in df_HRMUA.iterrows()]
#HRMUA_texts, HRMUA_langs = get_twts_from_list_account_names(HRMUA_flwrs, max_num_followers=300)

In [66]:
df_txts_langs_HRMUA = pd.DataFrame({'texts':HRMUA_texts, 'lang':HRMUA_langs})

df_txts_langs_HRMUA.to_hdf('lang_data.h5', '/ukr_nodes/news/HromadskeUA/tls_followers')

In [ ]:
df_txts = pd.read_hdf('lang_data.h5', '/ukr_nodes/news/HromadskeUA/tls_followers')

### CATALONIA NODES

In [125]:
Catalonia_nodes = {}
Catalonia_nodes['news'] = ['LaVanguardia', 'VilaWeb', 'diariARA', 'elperiodico',
                           'elperiodico_cat', 'elpuntavui']

In [126]:
country = 'cat'
node_name = 'news'
acc_name = 'elperiodico'
rel_type = 'followers'

path_save = '/'.join(['',country, node_name, acc_name, rel_type])
followers = get_account_network(acc_name, rel_type=rel_type, 
                                max_num =5000, key_words=None)
json_format = [elem._json for elem in followers]
df = pd.DataFrame(json_format)
df.to_hdf('lang_data.h5', path_save)

In [15]:
key_words=['Catal', 'Bcn', 'Barcel']
followers = get_account_network(screen_name, rel_type='followers', 
                                max_num =5000, key_words=None)
json_info = [elem._json for elem in followers]
df = pd.DataFrame(json_info)
df.to_hdf('lang_data.h5', path_save)

In [105]:
ARA_followers = get_account_network('diariARA', rel_type='followers', 
                                       max_num =5000, key_words=None)
json_info_ARA = [elem._json for elem in ARA_followers]
df_ARA = pd.DataFrame(json_info_ARA)
df_ARA.to_hdf('lang_data.h5', '/cat_nodes/news/diariARA/followers')

In [52]:
df_ARA_follwrs = pd.read_hdf('lang_data.h5', '/cat_nodes/news/diariARA/followers')

In [56]:
df_ARA_follwrs[df_ARA_follwrs['statuses_count'] > 5][['screen_name', 'statuses_count','followers_count']].count()
df_ARA_follwrs['lang'][df_ARA_follwrs['statuses_count'] > 5][df_ARA_follwrs['location'].str.contains(r"(Barcel|Catal)")].value_counts()

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app


es       240
ca       178
en        41
it         1
fr         1
en-GB      1
Name: lang, dtype: int64

In [83]:
counts_ARA_Cat = df_ARA['lang'][df_ARA['location'].str.contains(r"(Barcel|Catal)")].value_counts()

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


In [99]:
df_ARA['lang'][df_ARA['location'].str.contains(r"(Barcel|Catal|Tarr|Llei|Ger|Gir|Badal)")].count()/5000

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


0.13139999999999999

In [76]:
df_LaVang[['lang', 'screen_name']][df_LaVang['location'].str.contains(r"(Barcel|Catal)")]

### GET TWEETS FROM FOLLOWERS

In [260]:
country = 'ukr'
node_name = 'starsystem'
screen_name = 's_vakarchuk'
rel_type1 = 'followers'
rel_type2 = 'tls_followers'
min_num_twts_per_acc=5

path_load = '/'.join(['',country, node_name, screen_name, rel_type1])
path_save = '/'.join(['',country, node_name, screen_name, rel_type2])

key_words = {'ukr':r"(Україна|Ukraine|Украина|Київ|Киев|Львів|Одес)", 
             'cat':r"(Barcel|Catal|Tarr|Llei|Ger|Gir|Badal)"}
df = pd.read_hdf('lang_data.h5', path_load)
relevant_followers = df['screen_name'][df['statuses_count'] >= min_num_twts][
                         df['location'].str.contains(key_words[country])
                       ].values
relevant_followers.shape

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:15: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


(79,)

In [264]:
path_load

'/ukr/starsystem/s_vakarchuk/followers'

In [267]:
def get_tl_followers(screen_name, country, node_name, min_num_twts_per_acc=5, max_num_followers=None):
    """ Creates pandas dataframe with all tweet texts and corresponding language
    from followers of a given account. A dataframe with all followers info 
    must have been previously computed and saved in hdf5 format"""
    base_path = '/'.join(['',country, node_name, screen_name])
    path_load = base_path + '/followers'
    path_save = base_path + '/tls_followers'
    key_words = {'ukr':r"(Україна|Ukraine|Украина|Київ|Киев|Львів|Одес)", 
                 'cat':r"(Barcel|Catal|Tarr|Llei|Ger|Gir|Badal)"}
    df = pd.read_hdf('lang_data.h5', path_load)
    # filter by num_min_twts_per_account
    relevant_followers = df['screen_name'][df['statuses_count'] >= min_num_twts_per_acc]
    # keep only country residents
    relevant_followers = relevant_followers[df['location'].str.contains(key_words[country])].values
    texts, langs, auth = get_twts_from_list_account_names(relevant_followers, 
                                                          max_num_accounts=max_num_followers)
    df_txts_langs= pd.DataFrame({'texts':texts, 'lang':langs, 'screen_name':auth})
    df_txts_langs.to_hdf('lang_data.h5', path_save)
    return df_txts_langs
    
    

In [272]:
base_path = '/'.join(['',country, node_name, screen_name])
path_load = base_path + '/followers'
path_save = base_path + '/tls_followers'

In [280]:
df = pd.read_hdf('lang_data.h5', path_save)

In [284]:
df

,lang,screen_name,texts
0,en,ChristinaVaAn,"Hi, my name is Tina✋. I'm 16 years old and i'm..."
1,en,ChristinaVaAn,It's my brother's birthday today!!! Yeeeeeey)
2,en,ChristinaVaAn,When you have nothing to do https://t.co/vdK2e...
3,uk,ChristinaVaAn,@SolomiaNB Звучить по-філософськи
4,ru,ChristinaVaAn,@IloveLHMelovin Подивись
5,uk,ChristinaVaAn,@SolomiaNB А це вже нагло
6,en,ChristinaVaAn,RT @KatieStrope96: Someone please take advanta...
7,uk,ChristinaVaAn,@SolomiaNB Вільна(мрій)
8,ru,ChristinaVaAn,@SolomiaNB От +
9,ru,ChristinaVaAn,"@Brizer_06 Ееееей, все добре?"


In [230]:
# filter followers to focus on most relevant ones
min_num_twts = 5
relevant_followers = df['screen_name'][df['statuses_count'] >= min_num_twts][
                         df['location'].str.contains(key_words[country])
                       ].values

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [220]:
# get twts, lang, authors. Transform to pandas df and save
texts, langs, auth = get_twts_from_list_account_names(relevant_followers, max_num_followers=79)
df_txts_langs= pd.DataFrame({'texts':texts, 'lang':langs, 'screen_name':auth})
df_txts_langs.to_hdf('lang_data.h5', path_save)

0%                          100%
[#####                         ] | ETA: 00:01:51

Twitter error response: status code = 401
Twitter error response: status code = 401

[#########                     ] | ETA: 00:01:37


Twitter error response: status code = 401

[#############                 ] | ETA: 00:01:22

...


Twitter error response: status code = 401

[########################      ] | ETA: 00:00:33


Twitter error response: status code = 401

[##############################] | ETA: 00:00:00


Total time elapsed: 00:02:40


In [122]:
#df_txts_langs['lang'].value_counts()
#langs_detected = [detect(txt) for txt in df_txts_langs['texts']]

langs_detected=[]
for txt in df_txts_langs['texts']:
    try:
        langs_detected.append(detect(txt))
    except:
        langs_detected.append(None)

In [55]:
df_try = pd.DataFrame({'a':['aaaa','bbbfdde',1],'b':[23,44,56]})
df_try2 = pd.DataFrame({'a':['xxxx','zzzz'],'b':[3233,43214]})

store = pd.HDFStore('try_hyerar.h5')

store.append('city/topic', df_try)

store.close()

pd.read_hdf('try_hyerar.h5', 'city/topic')

store = pd.HDFStore('try_hyerar.h5','a')

store.append('city/topic', df_try2)

store.close()

pd.read_hdf('try_hyerar.h5', 'city/topic')

store = pd.HDFStore('try_hyerar.h5','a')

store.put('city/followers',df)

store.close()

### MY ACCOUNT

In [19]:
api = tweepy.API(auth)
user_info = api.get_user('ArnauAndreu')  
user_info._json['lang']

'ca'

In [6]:
user_info._json['location']

'Ukraine'

In [324]:
my_friends = get_account_network('ArnauAndreu')

my_df = pd.DataFrame(my_followers)

Counter([friend.lang for friend in my_friends])

my_fr_txts, my_friends_lang = get_twts_from_list_account_names(my_friends)

#Counter(my_friends_lang)



# my_fr_langs_detected=[]
# for i,txt in enumerate(my_fr_txts):
#     #print(i, txt)
#     try:
#         my_fr_langs_detected.append(detect(txt))
#     except:
#         continue

#Counter(my_fr_langs_detected)

usr_tl = get_account_timeline(my_friends[44].screen_name, max_num=10)

df_try=pd.DataFrame([twt._json for twt in usr_tl])
df_try.columns

my_tl = get_account_timeline('ArnauAndreu', max_num=10)

### DYNAMO KIEV PLAYERS

In [56]:
# DYNAMO PLAYERS

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

url = "http://www.worldfootball.net/teams/dinamo-kiev/2017/2/"
html = requests.get(url)
soup = BeautifulSoup(html.text, "lxml")

dynamo_players = pd.read_html(url, encoding='utf8')[1][2].dropna().values

dynamo_players

dyn_play_countr = pd.read_html(url, encoding='utf8')[1][[2,4]].dropna().values

dyn_play_countr

### PYMONGO

In [5]:
from pymongo import MongoClient

conn=MongoClient()

#define database
db = conn.citylangs

#define collection inside database
collection = db.bcn.ajuntam.followers

# function to add documents to collection 
def make_followers_collection(account_name, collection, max_num=100):
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    users = tweepy.Cursor(api.followers, screen_name=account_name).items(max_num)
    #i = 0
    while True:
        try:
            user = next(users)
            collection.insert_one(user._json)
        except tweepy.TweepError as e:
            if 'Read timed out' in str(e):
                print('fall here')
                print(e)
                time.sleep(5)
            else:
                time.sleep(60*16)
                user = next(users)
        except StopIteration:
            break
        #print ("@" + user.screen_name)
    #return collection

make_followers_collection('bcn_ajuntament',collection, max_num=100)

#check what databases are available
conn.database_names()

# available collections inside db
db.collection_names()

collection2 = db.kiev.ukrpravda

make_followers_collection('ukrpravda_news',collection2, max_num=300)

l = list(conn.citylangs.kiev.ukrpravda.find())

Counter([obj['lang'] for obj in l])


db.collection_names()

#db.categories.insert_one({ "_id": "ukr_pravda", "children": [] })
# db.categories.insert({ _id: "avto_kiev", children: [] })
# db.categories.insert({ _id: "kiev", children: ["ukr_pravda", "avto_kiev"] })

db['kiev'].insert_one({'avto_kiev':[],'vitklitschko':[]})

db.collection_names()

rr=list(db['bcn.ajuntam.followers'].find())

db['kiev'].find_one()

db['bcn.ajuntam.followers']

conn.database_names()

coll2 = db.countries

### MERGE DFs

In [244]:
#get hdf database keys
with pd.HDFStore('lang_data.h5','r') as f:
    my_keys = f.keys()

In [245]:
my_keys

['/ukr_nodes/news/BBC_ua/followers',
 '/ukr_nodes/news/BBC_ua/tls_followers',
 '/ukr_nodes/news/HromadskeUA/followers',
 '/ukr_nodes/news/HromadskeUA/tls_followers',
 '/ukr_nodes/news/LIGAnet/followers',
 '/ukr_nodes/news/LIGAnet/tls_followers',
 '/ukr_nodes/news/ukrpravda_news/followers',
 '/cat_nodes/news/LaVanguardia/followers',
 '/cat_nodes/news/LaVanguardia/tls_followers',
 '/cat_nodes/news/diariARA/followers',
 '/cat_nodes/news/diariARA/tls_followers']

In [164]:
file_path = 'lang_data.h5'
country = 'cat'
acc_names = ['diariARA', 'LaVanguardia']
load_node1 = '/' + country + '_nodes/news/' + acc_names[0] + '/tls_followers'
load_node2 = '/' + country + '_nodes/news/' + acc_names[1] + '/tls_followers'

df1 = pd.read_hdf(file_path, load_node1)
df2 = pd.read_hdf(file_path, load_node2)

In [165]:
df_merged = pd.merge(df1, df2, how='outer')

In [166]:
df_merged.shape, df1.shape, df2.shape

((44937, 2), (34461, 2), (12181, 2))

In [180]:
%matplotlib
from matplotlib import pyplot as plt


Using matplotlib backend: MacOSX


FACEBOOK

In [ ]:
fb_token = 'EAADT8MeSHbsBAPi29inYYj1xZBsywhwduhm5XZA1jI07qzmPA7OLVPZBBtRDCPv7LTyUvhxWut5ztyZAS5lvNMUU'